### **Import Dependencies**

In [1]:
import os
from getpass import getpass
from IPython.display import display, Markdown
from astrapy.info import CollectionVectorServiceOptions
from langchain_astradb import AstraDBVectorStore
from langchain_community.document_loaders import CSVLoader
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

### **Initialize the VectorStore**

In [2]:
ASTRA_DB_API_ENDPOINT = "Your AstraDB API Endpoint"
ASTRA_DB_APPLICATION_TOKEN = "Your AstraDB Application Token"

desired_namespace = input("(optional) Namespace = ")
if desired_namespace:
    ASTRA_DB_KEYSPACE = desired_namespace
else:
    ASTRA_DB_KEYSPACE = None

In [3]:
openai_api_key = "Your Openai API Key"
embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [4]:
vstore = AstraDBVectorStore(
    embedding=embeddings,
    collection_name="your collection name",
    api_endpoint=ASTRA_DB_API_ENDPOINT,
    token=ASTRA_DB_APPLICATION_TOKEN,
    namespace=ASTRA_DB_KEYSPACE,
)

### **Load a dataset**

In [5]:
# Load CSV Data
file_path = 'path to data'
def load_data(file_path):
    loader = CSVLoader(file_path=file_path)
    data = loader.load()
    return data

data = load_data(file_path=file_path)
print(f"Number of records loaded: {len(data)}")

Number of records loaded: 5


In [6]:
# Inspect the first entry to understand its structure
print(data[0])

page_content='Country: Saudi Arabia
City: Riyadh
Event_Name: Unfolding the Embassy
Description: ‘‘Unfolding the Embassy ‘‘presents a fictitious time-space, a rediscovery of the human condition in the very present of 2024, through the perceptual distance of the dauntingly near future.
The date is 2040, and we are situated in a Space-X satellite orbiting around a meta-earth, decomposed of its state borders due to environmental failure, and regulated by cyborgs wary of human intervention.
‘‘Unfolding the Embassy ‘‘brings together works of installation, video, photography, and print media that address the state of our globe in relation to the economy, the Anthropocene, and artificial intelligence.
In exploring the works of art and their conversations with one another, exhibition visitors are invited to speculate on the role of fiction in constructing the systems that hold our societies together.
Here, decontextualized geographic borders, language symbols, and national emblems trigger audie

### **Insert the Data into DB**

In [7]:
# Function to create metadata automatically
def create_metadata(doc):
    metadata = {}
    for key, value in doc.metadata.items():
        metadata[key.lower()] = value
    return metadata

In [8]:
# Prepare and insert documents directly
docs = []
for doc in data:
    metadata = create_metadata(doc)
    page_content = doc.page_content
    new_doc = Document(page_content=page_content, metadata=metadata)
    docs.append(new_doc)


In [9]:
# Insert documents into the vector store
inserted_ids = vstore.add_documents(docs)
print(f"\nInserted {len(inserted_ids)} documents.")


Inserted 5 documents.


### **Run Searches**

In [10]:
query = "Most expensive event in Riyadh"
results = vstore.similarity_search_with_relevance_scores(query, k=1)
for res, score in results:
    print(f"* [Relevance Score={score:3f}]\n{res.page_content} [{res.metadata}]")

* [Relevance Score=0.915334]
Country: Saudi Arabia
City: Riyadh
Event_Name: AFT_R
Description: Get ready for AFT_r, The Esports World Cup After show in Riyadh. Enjoy international, regional, and local artists every week after the Esports World Cup competitions. Get your tickets and join the fun.
Category: Concerts & Shows
Start_Date: 04-Jul-24
End_Date: 15-Aug-24
Open_Days: Daily
Open_Hours: 8:00 PM - 2:30 AM
Price_Starting_From: 149 SAR
Per Person
Direct_Booking: Yes
Links: https://webook.com/en/events/after-is-here-mdlbeast-tickets/book
Ages: All
Tags: Concerts & Shows,
Families,
Friends
Terms_and_Conditions: Signup For more information.
Location: QJC4+XH4 Mohamed Abdo Theater, Hittin, Riyadh 13516, Saudi Arabia
Guide_Manual: Visit website for more information [{'source': './data/Riyadh_Events.csv', 'row': 2}]


### **RAG Chain**

In [11]:
retriever = vstore.as_retriever(search_kwargs={"k":3})

template = """
    You are an intelligent chatbot that answers the users queries \
    based on the context from the data from the database. \
    Use the provided context as the basis for your answers \
    and do not make up new reasoning paths - just mix-and-match what you are given.
    Your answers must be concise and to the point, \
    and refrain from answering about other topics.

    CONTEXT:
    {context}

    QUESTION: {question}

    YOUR ANSWER: 
"""
prompt = ChatPromptTemplate.from_template(template)


In [12]:
# Initialize the LLM
llm = ChatOpenAI(
    api_key=openai_api_key,
    model='gpt-4-turbo',
    temperature=0.0,
)

**Chain**

In [13]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

### **Testing**

In [14]:
question = "When does the AFT_r event starts?"
chain.invoke(question)

'The AFT_r event starts on the 4th of July, 2024.'

In [15]:
question = input("Hello, How may I help you: ")
res = chain.invoke(question)
display(Markdown(res))

The best time to visit Riyadh for specific events would be during the Otaiqah Dates Season 2024, which runs from August 1, 2024, to November 3, 2024, or during the AFT_R event, which is from July 4, 2024, to August 15, 2024. These events offer unique experiences such as the largest dates season and an exciting Esports World Cup aftershow with concerts and shows.

In [16]:
question = "Give me a summary of each event in Riyadh in a tabular format"
res = chain.invoke(question)
display(Markdown(res))

| Event Name            | Category          | Description                                                                                                                                                                                                                                                                                                                                                                                                                                                                 | Start Date | End Date   | Location                                         |
|-----------------------|-------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|------------|------------|--------------------------------------------------|
| Unfolding the Embassy | Culture & History | Presents a fictitious time-space set in 2040, exploring human condition through art installations, video, photography, and print media. It addresses global issues like the economy, the Anthropocene, and artificial intelligence, inviting visitors to speculate on the role of fiction in societal systems.                                                                                                                                                                   | 16-May-24  | 01-Sep-24  | Fenaa Al Awwal                                  |
| AFT_R                 | Concerts & Shows  | The Esports World Cup Aftershow featuring international, regional, and local artists performing weekly. This event combines the excitement of esports with live music performances.                                                                                                                                                                                                                                                                                           | 04-Jul-24  | 15-Aug-24  | Mohamed Abdo Theater, Hittin, Riyadh 13516, Saudi Arabia |

In [19]:
question = input("Hello, How may I help you: ")
res = chain.invoke(question)
display(Markdown(res))

Hello! How can I assist you today?